In [28]:
# Imports
load('prismatic_envelope.sage')
load('precision.py')

### User-defined input

In [29]:
# The prime p
p=2

# The motivic weight i in F_p(i)^red
i=3

# The power of the uniformizer n
n=2

# The residual degree f
# The present code only supports totally ramified extensions of Qp,
# i.e., where f=1
f=1

### Some precision calculations

In [30]:
# The calculated F-precision needed to compute at this weight
Fprec=n*i

# The target precision
target_precision=nygaard_exponent(p,i,n)

####################
# Precision losses #
####################

### From \delta
precision_loss_delta=math.floor(math.log(Fprec-1,p))

### From passing from OK to OK/pi^n
precision_loss_quotient=0
for q in range(p,i):
    precision_loss_quotient+=n*valuation(p,math.factorial(q))
    
### From lifting nabla to Nygaard
precision_loss_nygaard=n*math.floor(i*(i-1)/2)

### From computing can-phi on primitives
precision_loss_primitives=target_precision

### From renormalizing the Eisenstein polynomial
precision_loss_from_Eisenstein=1

### Probably this precision can be taken to be lower since we will only need the
### Fp-coefficient calculation
total_precision=(target_precision+precision_loss_delta
                 +precision_loss_quotient
                 +precision_loss_nygaard
                 +precision_loss_primitives
                 +precision_loss_from_Eisenstein)

print("total p-adic precision is {}".format(total_precision))
print("Fprec is {}".format(Fprec))

# The coefficient ring W
if f==1:
    W=Zp(p,total_precision,type='capped-abs',print_mode='digits',show_prec=False)
else:
    W=Zq(p**f,total_precision,names='a',type='capped-abs',print_mode='series',show_prec=False)
    
# The Breuil-Kisin ring A
A.<z>=PowerSeriesRing(W,Fprec)

total p-adic precision is 21
Fprec is 6


### User-defined input: the Eisenstein polynomial

Note that in order for this to be created in a power series ring with the correct p-adic and F-adic precisions,
the elements p,i,n,f should be set above *before* defining the Eisenstein polynomial.

In [31]:
# The Eisenstein polynomial E
E=A(z+p)

### The main calculation

In [32]:
#%%capture
# Suppresses some Python warnings and SAGE variable injections

# The normalized Eisenstein polynomial
# The normalization is to bring the Eisenstein polynomial into the form E(0)=p
E=eisenstein_normalization(p,E)

if i-p+1>0:
    a_syn0,a_syn1,a_nablaN,a_nablaP=syntomic_matrices(p,i-p+1,n,E,total_precision,Fprec,debug=False)
    b_syn0,b_syn1,b_nablaN,b_nablaP=syntomic_matrices(p,i,n,E,total_precision,Fprec,debug=False)

<string>:672: DeprecationWarning: __getitem__ is changing to match the behavior of number fields. Please use expansion instead.
See https://github.com/sagemath/sage/issues/14825 for details.
<string>:680: DeprecationWarning: __getitem__ is changing to match the behavior of number fields. Please use expansion instead.
See https://github.com/sagemath/sage/issues/14825 for details.
<string>:697: DeprecationWarning: __getitem__ is changing to match the behavior of number fields. Please use expansion instead.
See https://github.com/sagemath/sage/issues/14825 for details.
<string>:705: DeprecationWarning: __getitem__ is changing to match the behavior of number fields. Please use expansion instead.
See https://github.com/sagemath/sage/issues/14825 for details.
<string>:719: DeprecationWarning: __getitem__ is changing to match the behavior of number fields. Please use expansion instead.
See https://github.com/sagemath/sage/issues/14825 for details.
<string>:727: DeprecationWarning: __getitem__

### Assembling the syntomic complex and computing its cohomology

In [33]:
# The K-groups (cohomology of the p-adic syntomic complex)
# New
coh_dict,final_precision=syntomic_cohomology(a_syn0,a_syn1,a_nablaN,a_nablaP)

print('Elementary divisors of K_{}(R;Z_p)'.format(2*(i-p+1)-2)+' are {}'.format(coh_dict['h2'][1]))
print('Elementary divisors of K_{}(R;Z_p)'.format(2*(i-p+1)-1)+' are {}'.format(coh_dict['h1'][1]))
print('Target precision is {} and final precision is {}'.format(target_precision,final_precision))

Elementary divisors of K_2(R;Z_p) are [10]
Elementary divisors of K_3(R;Z_p) are [1000]
Target precision is 5 and final precision is 15


In [34]:
# The K-groups (cohomology of the p-adic syntomic complex)
# New
coh_dict,final_precision=syntomic_cohomology(b_syn0,b_syn1,b_nablaN,b_nablaP)

print('Elementary divisors of K_{}(R;Z_p)'.format(2*i-2)+' are {}'.format(coh_dict['h2'][1]))
print('Elementary divisors of K_{}(R;Z_p)'.format(2*i-1)+' are {}'.format(coh_dict['h1'][1]))
print('Target precision is {} and final precision is {}'.format(target_precision,final_precision))

Elementary divisors of K_4(R;Z_p) are []
Elementary divisors of K_5(R;Z_p) are [1000]
Target precision is 5 and final precision is 13


In [35]:
a_d0=block_matrix([[a_syn0],[a_nablaN]])
a_d1=block_matrix([[a_nablaP,-a_syn1]])
a_d0_Fp=Matrix(GF(p),a_d0)
a_d1_Fp=Matrix(GF(p),a_d1)
a_C=ChainComplex({0:a_d0_Fp,1:a_d1_Fp})
a_C.homology()

{0: Vector space of dimension 1 over Finite Field of size 2,
 1: Vector space of dimension 2 over Finite Field of size 2,
 2: Vector space of dimension 1 over Finite Field of size 2}

In [36]:
b_d0=block_matrix([[b_syn0],[b_nablaN]])
b_d1=block_matrix([[b_nablaP,-b_syn1]])
b_d0_Fp=Matrix(GF(p),b_d0)
b_d1_Fp=Matrix(GF(p),b_d1)
b_C=ChainComplex({0:b_d0_Fp,1:b_d1_Fp})
b_C.homology()

{0: Vector space of dimension 1 over Finite Field of size 2,
 1: Vector space of dimension 1 over Finite Field of size 2,
 2: Vector space of dimension 0 over Finite Field of size 2}

In [37]:
ascii_art(a_C)

                                [0 0 0]      
                                [1 0 0]      
                                [1 1 0]      
            [1 0 0 0 0 0]       [0 0 0]      
            [0 0 0 0 0 0]       [1 0 0]      
            [0 0 1 0 1 1]       [0 1 0]      
 0 <-- C_2 <-------------- C_1 <-------- C_0 <-- 0 

In [38]:
ascii_art(b_C)

                                        [0 0 0 0 0]      
                                        [1 0 0 0 0]      
                                        [0 0 0 0 0]      
                                        [0 1 0 0 0]      
                                        [0 0 0 1 0]      
            [1 0 0 0 0 0 0 0 0 0]       [0 0 0 0 0]      
            [0 0 0 0 0 1 0 0 0 0]       [0 0 0 0 0]      
            [1 0 1 0 0 1 0 0 0 0]       [0 0 0 0 0]      
            [0 0 0 0 0 1 0 1 0 0]       [0 0 1 0 0]      
            [0 0 0 0 1 0 0 0 0 1]       [0 0 0 1 0]      
 0 <-- C_2 <---------------------- C_1 <------------ C_0 <-- 0 

# Computing v1

In [39]:
v1N0,v1P0,v1N1,v1P1=v1_matrices(p,i,n,E,total_precision,Fprec,debug=False)

fprod and c are 1 and 0
input is d_tilde^3
reduced_form is (10 + z + O(z^6))*d_tilde^2
10 + z + O(z^6)


gprod and a are 1 and 0
coefficient to process is 10 + z + O(z^6)
gprod and a are 1 and 1
coefficient to process is 10 + z + O(z^6)
gprod and a are f0 and 0
coefficient to process is 0
gprod and a are f0 and 1
coefficient to process is 0
gprod and a are f1 and 0
coefficient to process is 0
fprod and c are 1 and 1
input is z*d_tilde^3
reduced_form is (10*z + O(z^6))*d_tilde^2 + ((10 + z + O(z^4))*f0)*d_tilde
(10 + z + O(z^4))*f0 + 10*z + O(z^6)


gprod and a are 1 and 0
coefficient to process is 10*z + O(z^6)
gprod and a are 1 and 1
coefficient to process is 10*z + O(z^6)
gprod and a are f0 and 0
coefficient to process is 10 + z + O(z^4)
gprod and a are f0 and 1
coefficient to process is 10 + z + O(z^4)
gprod and a are f1 and 0
coefficient to process is 0
fprod and c are f0 and 0
input is f0*d_tilde^2
reduced_form is ((10 + z + O(z^4))*f0)*d_tilde
(10 + z + O(z^4))*f0


gprod and a a

<string>:801: DeprecationWarning: __getitem__ is changing to match the behavior of number fields. Please use expansion instead.
See https://github.com/sagemath/sage/issues/14825 for details.
<string>:810: DeprecationWarning: __getitem__ is changing to match the behavior of number fields. Please use expansion instead.
See https://github.com/sagemath/sage/issues/14825 for details.
<string>:822: DeprecationWarning: __getitem__ is changing to match the behavior of number fields. Please use expansion instead.
See https://github.com/sagemath/sage/issues/14825 for details.
<string>:834: DeprecationWarning: __getitem__ is changing to match the behavior of number fields. Please use expansion instead.
See https://github.com/sagemath/sage/issues/14825 for details.
<string>:846: DeprecationWarning: __getitem__ is changing to match the behavior of number fields. Please use expansion instead.
See https://github.com/sagemath/sage/issues/14825 for details.
<string>:855: DeprecationWarning: __getitem__

In [40]:
v1N0_Fp=Matrix(GF(p),v1N0)
v1P0_Fp=Matrix(GF(p),v1P0)
v1N1_Fp=Matrix(GF(p),v1N1)
v1P1_Fp=Matrix(GF(p),v1P1)

In [41]:
v1N0_Fp

[0 0 0]
[0 0 0]
[1 1 0]
[0 0 0]
[0 0 0]

In [42]:
v1N1_Fp

[0 0 0]
[1 0 0]
[0 0 0]
[0 1 1]
[0 0 0]

In [43]:
v1P1_Fp

[0 0 0]
[0 0 0]
[1 0 0]
[0 1 0]
[0 0 0]

In [18]:
v1_0=Matrix(GF(p),v1N0)
v1_1=Matrix(GF(p),block_matrix([[v1P0,0],[0,v1N1]]))
v1_2=Matrix(GF(p),v1P1)

In [19]:
v1_0

[0 0 0]
[0 0 0]
[1 1 0]
[0 0 0]
[0 0 0]

In [20]:
v1_1

[0 0 0 0 0 0]
[0 0 0 0 0 0]
[1 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 1 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]

In [21]:
v1_2

[0 0 0]
[0 0 0]
[1 0 0]
[0 1 0]
[0 0 0]

In [22]:
class Homology():
    def __init__(self,C):
        self.complex = C
        self.compute_homology()
    def compute_homology(self):
        self.complex_smith_form()
        nz=self.complex.nonzero_degrees()
        deg_min=min(nz)
        deg_max=max(nz)
        self.homology_orders={}
        self.homology_representatives={}
        for i in range(deg_min,deg_max+1):
            self.homology_orders[i] = self.homology_smith_form(i)
            self.homology_representatives[i] = self.compute_homology_representatives(i)
        
    def complex_smith_form(self):
        # Returns a triple (D,f,g) where each differential is in ``Smith form''
        # and f is a map of chain complexes C -> D.
        # Assumes that the degree of the differential is +1.
        nz=self.complex.nonzero_degrees()
        deg_min=min(nz)
        deg_max=max(nz)
        # The number of differentials
        lt=deg_max-deg_min
        self.f_dict={}
        self.g_dict={}
        for i in range(deg_min,deg_max+1):
            self.f_dict[i]=identity_matrix(self.complex.free_module_rank(i))
            self.g_dict[i]=identity_matrix(self.complex.free_module_rank(i))
        D_dict={}
        j=0
        S,U,V=self.complex.differential()[deg_max-(j+1)].smith_form()
        D_dict[deg_max-(j+1)]=S
        self.f_dict[deg_max-j]=U*self.f_dict[deg_max-j]
        self.f_dict[deg_max-(j+1)]=V^(-1)*self.f_dict[deg_max-(j+1)]
        self.g_dict[deg_max-j]=self.g_dict[deg_max-j]*U^(-1)
        self.g_dict[deg_max-(j+1)]=self.g_dict[deg_max-(j+1)]*V    
        for j in range(1,lt):
            new_d=self.f_dict[deg_max-j]*self.complex.differential()[deg_max-(j+1)]
            col_offset=0
            row_offset=D_dict[deg_max-j].transpose().rank()
            col_num=self.complex.differential()[deg_max-(j+1)].dimensions()[1]
            row_num=D_dict[deg_max-j].transpose().nullity()
            new_d_sub=new_d.submatrix(row_offset,col_offset,row_num,col_num)
            S_sub,U_sub,V=new_d_sub.smith_form()
            S=block_matrix([[Matrix(row_offset,col_num)],[S_sub]])
            U=block_matrix([[identity_matrix(row_offset),0],[0,U_sub]])
            self.f_dict[deg_max-j]=U*self.f_dict[deg_max-j]
            self.f_dict[deg_max-(j+1)]=V^(-1)*self.f_dict[deg_max-(j+1)]
            self.g_dict[deg_max-j]=self.g_dict[deg_max-j]*U^(-1)
            self.g_dict[deg_max-(j+1)]=self.g_dict[deg_max-(j+1)]*V
            D_dict[deg_max-(j+1)]=S
        self.complex_smith = ChainComplex(D_dict)
    def homology_smith_form(self,i):
        nz=self.complex_smith.nonzero_degrees()
        deg_min=min(nz)
        deg_max=max(nz)
        if i<deg_min or i>deg_max:
            return []
        elif i==deg_max:
            num_gens=self.complex_smith.differential()[i-1].nrows()
            offset=0
            l=[]
            for j in range(num_gens):
                if j<self.complex_smith.differential()[i-1].ncols() and j+offset<self.complex_smith.differential()[i-1].nrows():
                    div=self.complex_smith.differential()[i-1][j+offset,j]
                    if not div.is_unit():
                        l.append(div)
                else:
                    l.append(0)
            return l
        elif i>deg_min:
            num_gens=self.complex_smith.differential()[i].ncols()-self.complex_smith.differential()[i].rank()
            l=[]
            offset=self.complex_smith.differential()[i].rank()
            for j in range(num_gens):
                if j<self.complex_smith.differential()[i-1].ncols() and j+offset<self.complex_smith.differential()[i-1].nrows():
                    div=self.complex_smith.differential()[i-1][j+offset,j]
                    if not div.is_unit():
                        l.append(div)
                else:
                    l.append(0)
            return l
        elif i==deg_min:
            num_gens=self.complex_smith.differential()[i].ncols()-self.complex_smith.differential()[i].rank()
            l=[0]*num_gens
            return l
    def compute_homology_representatives(self,i):
        nz=self.complex_smith.nonzero_degrees()
        deg_min=min(nz)
        deg_max=max(nz)
        if i<deg_min or i>deg_max:
            return []
        h = self.homology_orders[i]
        return self.g_dict[i].submatrix(0,self.g_dict[i].ncols() - len(h))

class MorphismHomology():
    def __init__(self, hA, hB, F):
        self.hA = hA
        self.hB = hB
        self.F = F
        self.compute_morphisms()
    def compute_morphisms(self):
        nz=self.F.keys()
        deg_min=min(nz)
        deg_max=max(nz)
        self.hF = {}
        for i in range(deg_min, deg_max+1):
            new_F=self.hB.f_dict[i]*self.F[i]*self.hA.g_dict[i]
            x=len(self.hA.homology_orders[i])
            y=len(self.hB.homology_orders[i])
            self.hF[i] = new_F.submatrix(new_F.nrows()-y,new_F.ncols()-x)


In [23]:
v1={0:v1_0,1:v1_1,2:v1_2}
ha_C = Homology(a_C)
hb_C = Homology(b_C)
mor = MorphismHomology(ha_C, hb_C, v1)


for h in mor.hF.values():
    print(ascii_art(h))
    print('\n')

[0]


[0 1]


[]




In [24]:
ascii_art(ha_C.complex_smith)

                                [0 0 0]      
                                [0 0 0]      
                                [-----]      
                                [1 0 0]      
            [1 0 0 0 0 0]       [0 1 0]      
            [0 1 0 0 0 0]       [0 0 0]      
            [0 0 0 0 0 0]       [0 0 0]      
 0 <-- C_2 <-------------- C_1 <-------- C_0 <-- 0 

In [25]:
ascii_art(hb_C.complex_smith)

                                        [0 0 0 0 0]      
                                        [0 0 0 0 0]      
                                        [0 0 0 0 0]      
                                        [0 0 0 0 0]      
                                        [0 0 0 0 0]      
                                        [---------]      
            [1 0 0 0 0 0 0 0 0 0]       [1 0 0 0 0]      
            [0 1 0 0 0 0 0 0 0 0]       [0 1 0 0 0]      
            [0 0 1 0 0 0 0 0 0 0]       [0 0 1 0 0]      
            [0 0 0 1 0 0 0 0 0 0]       [0 0 0 1 0]      
            [0 0 0 0 1 0 0 0 0 0]       [0 0 0 0 0]      
 0 <-- C_2 <---------------------- C_1 <------------ C_0 <-- 0 

In [26]:
ha_C.homology_representatives[1]

[0 0]
[0 0]
[1 0]
[0 1]
[1 0]
[0 0]

In [27]:
ha_C.homology_orders[1]

[0, 0]